참고: https://untitledtblog.tistory.com/169
### 1.라이브러리 불러오기
API를 받아 왔다면 라이브러리를 불러온다.<br>
그 전에 Google API 사용이 처음이라면 pip 명령어를 이용하여 라이브러리를 다운받는다.<br>

```pip install google-api-python-client```

- pandas: 수집한 댓글을 엑셀의 형태로 저장하기 위한 라이브러리
- googleapiclient.discovery의 build: Google API를 호출하기 위한 객체를 지원하는 라이브러리

In [2]:
import pandas
from googleapiclient.discovery import build

- comments: 수집한 댓글 저장. <br>
- build 함수를 이용하여 Google API 객체를 생성.<br>
build 함수에 이용된 api_key에는 발급 받은 API 키를 입력.
- API 객체를 호출. <br>
 video_id에는 댓글을 수집하고자 하는 YouTube 동영상의 id를 입력. <br>
API를 호출하면 response 변수에 전달 받은 데이터가 저장된다.

In [8]:
comments = list()
api_obj = build('youtube', 'v3', developerKey='받은 api 입력')
response = api_obj.commentThreads().list(part='snippet,replies',videoId='divunuWDwqo', maxResults=100).execute()

#### 댓글을 저장하고 API를 반복적으로 호출. 
- textDisplay: 댓글의 내용
- authorDisplayName: 댓글 작성자
- publishedAt: 댓글 작성 시간
- likeCount: 좋아요 수

*6-10번째 줄에서는 댓글에 대한 댓글 (reply)이 있는 경우에 reply도 같이 수집하는 코드. 
마지막으로, 11~15번째 줄에서는 동영상에서 가져올 댓글이 아직 남은 경우에 nextPageToken을 기반으로 API를 다시 호출한다.
반복적인 API 호출을 통해 수집된 댓글은 앞에서 선언한 comments라는 변수에 저장된다.*

In [9]:
while response:
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
 
        if item['snippet']['totalReplyCount'] > 0:
            for reply_item in item['replies']['comments']:
                reply = reply_item['snippet']
                comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])
 
    if 'nextPageToken' in response:
        response = api_obj.commentThreads().list(part='snippet,replies', videoId='divunuWDwqo', pageToken=response['nextPageToken'], maxResults=100).execute()
    else:
        break


pandas 라이브러리를 이용하여 comments에 수집된 댓글을 엑셀 파일로 저장

In [14]:
df = pandas.DataFrame(comments)
df.to_csv('results.xlsx', header=['comment', 'author', 'date', 'num_likes'],  encoding = 'utf-8')